In [13]:
import pandas as pd
import numpy as np  

In [19]:
# importing data
df_male_2019 = pd.read_csv('../data/raw/open_2019_gender_1.csv')
df_male_2020 = pd.read_csv('../data/raw/open_2020_gender_1.csv')
df_male_2021 = pd.read_csv('../data/raw/open_2021_gender_1.csv')
df_male_2022 = pd.read_csv('../data/raw/open_2022_gender_1.csv')
df_male_2023 = pd.read_csv('../data/raw/open_2023_gender_1.csv')

/var/folders/bz/9r0tb8ds4cl78219hkb7t1lm0000gq/T/ipykernel_11740/1845967326.py:3: DtypeWarning: Columns (17,19,21,23,25) have mixed types. Specify dtype option on import or set low_memory=False.
  df_male_2020 = pd.read_csv('../data/raw/open_2020_gender_1.csv')
/var/folders/bz/9r0tb8ds4cl78219hkb7t1lm0000gq/T/ipykernel_11740/1845967326.py:6: DtypeWarning: Columns (18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df_male_2023 = pd.read_csv('../data/raw/open_2023_gender_1.csv')


# data preparation

In [20]:
#rename columns with year suffix
def rename_columns(df, year):
    for col in df.columns:
        if col.startswith('score_display_'):
            new_col_name = f'score_{year}_{col.split("_")[-1]}'
            df.rename(columns={col: new_col_name}, inplace=True)
        elif col.startswith('rank_'):
            new_col_name = f'rank_{year}_{col.split("_")[-1]}'
            df.rename(columns={col: new_col_name}, inplace=True)
        elif col.startswith('overallScore'):
            new_col_name = f'overall_score_{year}'
            df.rename(columns={col: new_col_name}, inplace=True)

In [21]:
rename_columns(df_male_2020, '2020')
rename_columns(df_male_2021, '2021')
rename_columns(df_male_2022, '2022')
rename_columns(df_male_2023, '2023')

In [38]:
#remove rows with sclaed or foundations scores

def contains_f_or_s(df, columns_to_check):
    return df[columns_to_check].astype(str).apply(lambda x: x.str.contains('- f|- s')).any(axis=1)


columns_to_check_2019 = df_male_2019.columns[1:]  
columns_to_check_2020 = df_male_2020.columns[1:]  
columns_to_check_2021 = df_male_2021.columns[1:] 
columns_to_check_2022 = df_male_2022.columns[1:] 
columns_to_check_2023 = df_male_2023.columns[1:] 

In [39]:
df_male_2019 = df_male_2019[~contains_f_or_s(df_male_2019, columns_to_check_2019)]
df_male_2020 = df_male_2020[~contains_f_or_s(df_male_2020, columns_to_check_2020)]
df_male_2021 = df_male_2021[~contains_f_or_s(df_male_2021, columns_to_check_2021)]
df_male_2022 = df_male_2022[~contains_f_or_s(df_male_2022, columns_to_check_2022)]
df_male_2023 = df_male_2023[~contains_f_or_s(df_male_2023, columns_to_check_2023)]

## merging dataframe

In [40]:
df = df_male_2023.merge(df_male_2022[['competitorId' ,'overall_score_2022', 'score_2022_1', 'score_2022_2', 'score_2022_3']], how='left', on='competitorId')
df = df.merge(df_male_2021[['competitorId' ,'overall_score_2021', 'score_2021_1', 'score_2021_2', 'score_2021_3', 'score_2021_4']], how='left', on='competitorId')
df = df.merge(df_male_2020[['competitorId' ,'overall_score_2020', 'score_2020_1', 'score_2020_2', 'score_2020_3', 'score_2020_4',  'score_2020_5']], how='left', on='competitorId')


In [41]:
df = df.set_index('competitorId')

## creating columns for speed and reps

In [42]:
pattern = r'(\d+)\s+reps|(\d+:\d+)'

df[['score_reps_2023_4', 'score_time_2023_4']] = df['score_2023_4'].str.extract(pattern)
df[['score_reps_2020_1', 'score_time_2020_1']] = df['score_2020_1'].str.extract(pattern)
df[['score_reps_2020_3', 'score_time_2020_3']] = df['score_2020_3'].str.extract(pattern)
df[['score_reps_2020_4', 'score_time_2020_4']] = df['score_2020_4'].str.extract(pattern)
df[['score_reps_2020_5', 'score_time_2020_5']] = df['score_2020_5'].str.extract(pattern)

In [43]:
#fill nan values with time cap or max repetition number

def fillna_custom(df, column, fill_value):
    df[column] = df[column].fillna(fill_value)

In [44]:
fillna_custom(df, 'score_reps_2023_4', 292)
fillna_custom(df, 'score_time_2023_4', '12:00')
fillna_custom(df, 'score_reps_2020_1', 180)
fillna_custom(df, 'score_time_2020_1', '15:00')
fillna_custom(df, 'score_reps_2020_3', 156)
fillna_custom(df, 'score_time_2020_3', '9:00')
fillna_custom(df, 'score_reps_2020_4', 240)
fillna_custom(df, 'score_time_2020_4', '20:00')
fillna_custom(df, 'score_reps_2020_5', 240)
fillna_custom(df, 'score_time_2020_5', '20:00')

In [45]:
#drop columns repeted 
df = df.drop(columns=['score_2023_4', 
                 'score_2020_1', 'score_2020_3', 'score_2020_4', 'score_2020_5'])

In [46]:
#rename columns for same pattern

df.rename(columns={ 'score_2023_1': 'score_reps_2023_1', 'score_2023_2': 'score_reps_2023_2', 'score_2023_3': 'score_reps_2023_3', 'score_2023_4': 'score_reps_2023_4'
                   , 'score_2022_1': 'score_reps_2022_1', 'score_2022_2': 'score_reps_2022_2', 'score_2022_3': 'score_reps_2022_3'
                   , 'score_2021_1': 'score_reps_2021_1', 'score_2021_2': 'score_reps_2021_2', 'score_2021_3': 'score_reps_2021_3', 'score_2021_4': 'score_reps_2021_4'
                   , 'score_2020_2': 'score_reps_2020_2'    }
                   , inplace=True)

## convert height and weight in same scale

In [47]:
def convert_height(height_str):
    if 'in' in height_str:
        inches = float(height_str.split()[0])
        cm = inches * 2.54
        return cm
    elif 'cm' in height_str:
        cm = float(height_str.split()[0])
        return cm
    else:
        return np.nan
    
def convert_weight(weight_str):
    if 'lb' in weight_str:
        lbs = float(weight_str.split()[0])
        kg = lbs / 2.20462
        return kg
    elif 'kg' in weight_str:
        kg = float(weight_str.split()[0])
        return kg
    else:
        return np.nan
    
def remove_letters(column_str):
    if 'lb' in column_str:
        lbs = int(column_str.split()[0])
        return lbs
    elif 'reps' in column_str:
        reps = int(column_str.split()[0])
        return reps
    elif ':' in column_str:
        minutes, seconds = map(int, column_str.split(':'))
        total_seconds = minutes * 60 + seconds
        return total_seconds
    else:
        try:
            number = int(column_str)
            return number
        except ValueError:
            return np.nan   

In [48]:
columns_to_convert = ['height', 'weight',
                        'score_reps_2023_1', 'score_reps_2023_2', 'score_reps_2023_3',
                        'score_time_2023_4', 'score_reps_2023_4',
                        'score_reps_2022_1', 'score_reps_2022_2', 'score_reps_2022_3', 
                        'score_reps_2021_1', 'score_reps_2021_2', 'score_reps_2021_3', 'score_reps_2021_4',
                        'score_reps_2020_1', 'score_time_2020_1', 'score_reps_2020_2',
                        'score_reps_2020_3', 'score_time_2020_3', 'score_reps_2020_4',
                        'score_time_2020_4', 'score_reps_2020_5', 'score_time_2020_5'
                        ]

df[columns_to_convert] = df[columns_to_convert].astype(str)

In [49]:
df['height'] = df['height'].apply(convert_height)
df['weight'] = df['weight'].apply(convert_weight)

df['score_reps_2023_1'] = df['score_reps_2023_1'].apply(remove_letters)
df['score_reps_2023_2'] = df['score_reps_2023_2'].apply(remove_letters)
df['score_reps_2023_3'] = df['score_reps_2023_3'].apply(remove_letters)
df['score_reps_2022_1'] = df['score_reps_2022_1'].apply(remove_letters)
df['score_reps_2022_2'] = df['score_reps_2022_2'].apply(remove_letters)
df['score_reps_2022_3'] = df['score_reps_2022_3'].apply(remove_letters)
df['score_reps_2021_1'] = df['score_reps_2021_1'].apply(remove_letters)
df['score_reps_2021_2'] = df['score_reps_2021_2'].apply(remove_letters)
df['score_reps_2021_3'] = df['score_reps_2021_3'].apply(remove_letters)
df['score_reps_2021_4'] = df['score_reps_2021_4'].apply(remove_letters)
df['score_reps_2020_2'] = df['score_reps_2020_2'].apply(remove_letters)

df['score_time_2023_4'] = df['score_time_2023_4'].apply(remove_letters)
df['score_reps_2023_4'] = df['score_reps_2023_4'].apply(remove_letters)

df['score_time_2020_1'] = df['score_time_2020_1'].apply(remove_letters)
df['score_time_2020_3'] = df['score_time_2020_3'].apply(remove_letters)
df['score_time_2020_4'] = df['score_time_2020_4'].apply(remove_letters)
df['score_time_2020_5'] = df['score_time_2020_5'].apply(remove_letters)
df['score_reps_2020_1'] = df['score_reps_2020_1'].apply(remove_letters)
df['score_reps_2020_3'] = df['score_reps_2020_3'].apply(remove_letters)
df['score_reps_2020_4'] = df['score_reps_2020_4'].apply(remove_letters)
df['score_reps_2020_5'] = df['score_reps_2020_5'].apply(remove_letters)

# data cleaning

In [209]:
df.isna().sum()

year                        0
competitorName              3
firstName                   1
lastName                    5
status                      0
gender                      0
age                         0
countryOfOriginCode        97
countryOfOriginName       528
regionId                    0
regionName                  0
affiliateId             16919
affiliateName           16919
height                  55372
weight                  53194
overallRank                 0
overall_score_2023          0
score_reps_2023_1       15808
score_reps_2023_2       17820
score_reps_2023_3       17799
rank_2023_1                 0
rank_2023_2                 0
rank_2023_3                 0
rank_2023_4                 0
overall_score_2022      63434
score_reps_2022_1       67683
score_reps_2022_2       69181
score_reps_2022_3       72739
overall_score_2021      85214
score_reps_2021_1       89199
score_reps_2021_2       90498
score_reps_2021_3       92554
score_reps_2021_4       92549
overall_sc

## countryOfOriginName

In [50]:
#data set error with missing country name
df.loc[(df.countryOfOriginName.isna()) & (df.regionName == 'Europe'), 'countryOfOriginName'] = 'Bealarus'
df.loc[(df.countryOfOriginName.isna()) & (df.regionName == 'Asia'), 'countryOfOriginName'] = 'Russia'

## weight and height

### removing outliers

In [51]:
df.describe()

,year,age,regionId,affiliateId,height,weight,overallRank,overall_score_2023,score_reps_2023_1,score_reps_2023_2,...,score_reps_2023_4,score_time_2023_4,score_reps_2020_1,score_time_2020_1,score_reps_2020_3,score_time_2020_3,score_reps_2020_4,score_time_2020_4,score_reps_2020_5,score_time_2020_5
count,134702.0,134702.000000,134702.000000,117783.000000,79330.000000,81508.000000,134702.000000,134702.000000,118894.000000,116882.000000,...,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000
mean,2023.0,34.751726,31.839297,15087.215167,186.678399,85.024216,71648.000549,287524.930513,175.733494,123.870048,...,171.166664,719.692195,174.330054,892.774057,139.081951,539.888302,222.634623,1199.880195,235.574594,1187.292609
std,0.0,7.857439,2.715004,9149.509368,662.306414,229.671738,45354.179799,157647.038954,21.818061,19.350880,...,78.450610,5.841817,14.957018,33.079739,34.436421,3.031637,34.815243,4.405068,11.773331,59.109607
min,2023.0,16.000000,28.000000,5.000000,1.000000,0.453593,1.000000,107.000000,1.000000,0.000000,...,1.000000,464.000000,1.000000,496.000000,1.000000,354.000000,1.000000,761.000000,1.000000,594.000000
25%,2023.0,29.000000,29.000000,6874.000000,174.000000,77.110795,33675.500000,160640.250000,168.000000,111.000000,...,125.000000,720.000000,180.000000,900.000000,156.000000,540.000000,240.000000,1200.000000,240.000000,1200.000000
50%,2023.0,34.000000,32.000000,15348.000000,178.000000,83.914688,67476.500000,281619.500000,180.000000,124.000000,...,139.000000,720.000000,180.000000,900.000000,156.000000,540.000000,240.000000,1200.000000,240.000000,1200.000000
75%,2023.0,40.000000,35.000000,22222.000000,182.880000,90.718582,103790.000000,395173.000000,184.000000,137.000000,...,224.000000,720.000000,180.000000,900.000000,156.000000,540.000000,240.000000,1200.000000,240.000000,1200.000000
max,2023.0,54.000000,35.000000,32333.000000,65535.000000,65535.000000,157742.000000,582777.000000,306.000000,191.000000,...,292.000000,720.000000,180.000000,900.000000,164.000000,540.000000,240.000000,1200.000000,240.000000,1200.000000


In [212]:
#removing outliers using word record values
df.loc[(df['height'] > 251) | (df['height'] < 83), 'height'] = pd.NA
df.loc[(df['weight'] > 595) | (df['weight'] < 21), 'weight'] = pd.NA

In [213]:
df.describe()

,year,age,regionId,affiliateId,height,weight,overallRank,overall_score_2023,score_reps_2023_1,score_reps_2023_2,score_reps_2023_3,rank_2023_1,rank_2023_2,rank_2023_3,rank_2023_4,overall_score_2022,score_reps_2022_1,score_reps_2022_2,score_reps_2022_3,overall_score_2021,score_reps_2021_1,score_reps_2021_2,score_reps_2021_3,score_reps_2021_4,overall_score_2020,score_reps_2020_2,score_reps_2023_4,score_time_2023_4,score_reps_2020_1,score_time_2020_1,score_reps_2020_3,score_time_2020_3,score_reps_2020_4,score_time_2020_4,score_reps_2020_5,score_time_2020_5
count,134702.0,134702.000000,134702.000000,117783.000000,78649.000000,81473.000000,134702.000000,134702.000000,118894.000000,116882.000000,116903.000000,134702.000000,134702.000000,134702.000000,134702.000000,71268.000000,67019.000000,65521.000000,61963.000000,49488.000000,45503.000000,44204.000000,42148.000000,42153.000000,35779.000000,32719.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000
mean,2023.0,34.751726,31.839297,15087.215167,178.556771,84.185133,71648.000549,287524.930513,175.733494,123.870048,193.994945,70923.101008,72718.374731,73165.744926,70717.709848,168337.802464,210.158403,142.307382,270.919258,191734.528492,303.786695,739.112614,252.572886,199.010296,220024.944996,500.835050,171.166664,719.692195,174.330054,892.774057,139.081951,539.888302,222.634623,1199.880195,235.574594,1187.292609
std,0.0,7.857439,2.715004,9149.509368,7.005217,11.484193,45354.179799,157647.038954,21.818061,19.350880,35.039180,44416.265384,45921.570161,45459.378833,43239.222649,101621.510195,49.737633,67.961690,190.331728,117611.573377,124.379861,373.602602,245.351308,37.940763,137772.190061,157.750006,78.450610,5.841817,14.957018,33.079739,34.436421,3.031637,34.815243,4.405068,11.773331,59.109607
min,2023.0,16.000000,28.000000,5.000000,83.000000,24.040424,1.000000,107.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,25.000000,1.000000,1.000000,1.000000,101.000000,1.000000,1.000000,1.000000,0.000000,64.000000,1.000000,1.000000,464.000000,1.000000,496.000000,1.000000,354.000000,1.000000,761.000000,1.000000,594.000000
25%,2023.0,29.000000,29.000000,6874.000000,174.000000,77.110795,33675.500000,160640.250000,168.000000,111.000000,169.000000,33679.000000,33827.000000,34185.000000,33690.000000,83048.000000,180.000000,115.000000,156.000000,96335.750000,216.000000,217.000000,135.000000,175.000000,107905.500000,403.000000,125.000000,720.000000,180.000000,900.000000,156.000000,540.000000,240.000000,1200.000000,240.000000,1200.000000
50%,2023.0,34.000000,32.000000,15348.000000,178.000000,83.914688,67476.500000,281619.500000,180.000000,124.000000,192.000000,67552.000000,69200.000000,69406.000000,67534.000000,163242.500000,211.000000,134.000000,167.000000,178699.500000,310.000000,879.000000,145.000000,198.000000,209366.000000,510.000000,139.000000,720.000000,180.000000,900.000000,156.000000,540.000000,240.000000,1200.000000,240.000000,1200.000000
75%,2023.0,40.000000,35.000000,22222.000000,182.880000,90.718582,103790.000000,395173.000000,184.000000,137.000000,217.000000,102871.000000,109393.000000,108625.000000,102939.000000,244744.500000,242.000000,154.000000,369.000000,269955.250000,381.000000,1040.000000,161.000000,225.000000,316621.000000,612.000000,224.000000,720.000000,180.000000,900.000000,156.000000,540.000000,240.000000,1200.000000,240.000000,1200.000000
max,2023.0,54.000000,35.000000,32333.000000,240.000000,542.043527,157742.000000,582777.000000,306.000000,191.000000,375.000000,152041.000000,148203.000000,148144.000000,134389.000000,391859.000000,390.000000,599.000000,719.000000,467118.000000,899.000000,1199.000000,899.000000,365.000000,548388.000000,1030.000000,292.000000,720.000000,180.000000,900.000000,164.000000,540.000000,240.000000,1200.000000,240.000000,1200.000000


In [214]:
df.isna().sum()

year                        0
competitorName              3
firstName                   1
lastName                    5
status                      0
gender                      0
age                         0
countryOfOriginCode        97
countryOfOriginName         0
regionId                    0
regionName                  0
affiliateId             16919
affiliateName           16919
height                  56053
weight                  53229
overallRank                 0
overall_score_2023          0
score_reps_2023_1       15808
score_reps_2023_2       17820
score_reps_2023_3       17799
rank_2023_1                 0
rank_2023_2                 0
rank_2023_3                 0
rank_2023_4                 0
overall_score_2022      63434
score_reps_2022_1       67683
score_reps_2022_2       69181
score_reps_2022_3       72739
overall_score_2021      85214
score_reps_2021_1       89199
score_reps_2021_2       90498
score_reps_2021_3       92554
score_reps_2021_4       92549
overall_sc

## fill nan with average by country

In [215]:
def fill_na_with_country_mean(df):
    mean_height_weight = df.groupby('countryOfOriginName')[['height', 'weight']].transform('mean')
    df[['height', 'weight']] = df[['height', 'weight']].fillna(mean_height_weight)
    return df

In [216]:
df = fill_na_with_country_mean(df)

In [186]:
df.isna().sum()

year                        0
competitorName              3
firstName                   1
lastName                    5
status                      0
gender                      0
age                         0
countryOfOriginCode        97
countryOfOriginName         0
regionId                    0
regionName                  0
affiliateId             16919
affiliateName           16919
height                      5
weight                     10
overallRank                 0
overall_score_2023          0
score_reps_2023_1       15808
score_reps_2023_2       17820
score_reps_2023_3       17799
rank_2023_1                 0
rank_2023_2                 0
rank_2023_3                 0
rank_2023_4                 0
overall_score_2022      63434
score_reps_2022_1       67683
score_reps_2022_2       69181
score_reps_2022_3       72739
overall_score_2021      85214
score_reps_2021_1       89199
score_reps_2021_2       90498
score_reps_2021_3       92554
score_reps_2021_4       92549
overall_sc

## replace values with std greather than 2

In [217]:
def replace_outliers_with_mean(df, column_name):
    # Calculate the mean by 'countryOfOriginName'
    mean_by_country = df.groupby('countryOfOriginName')[column_name].transform('mean')
    
    # Calculate the standard deviation by 'countryOfOriginName'
    std_by_country = df.groupby('countryOfOriginName')[column_name].transform('std')
    
    # Define a threshold for outliers 
    threshold = 2
    
    # Replace outliers with the mean
    df[column_name] = np.where(
        (df[column_name] < (mean_by_country - threshold * std_by_country)) |
        (df[column_name] > (mean_by_country + threshold * std_by_country)),
        mean_by_country,
        df[column_name]
    )

In [218]:
replace_outliers_with_mean(df, 'weight')
replace_outliers_with_mean(df, 'height')

In [219]:
df.describe()

,year,age,regionId,affiliateId,height,weight,overallRank,overall_score_2023,score_reps_2023_1,score_reps_2023_2,score_reps_2023_3,rank_2023_1,rank_2023_2,rank_2023_3,rank_2023_4,overall_score_2022,score_reps_2022_1,score_reps_2022_2,score_reps_2022_3,overall_score_2021,score_reps_2021_1,score_reps_2021_2,score_reps_2021_3,score_reps_2021_4,overall_score_2020,score_reps_2020_2,score_reps_2023_4,score_time_2023_4,score_reps_2020_1,score_time_2020_1,score_reps_2020_3,score_time_2020_3,score_reps_2020_4,score_time_2020_4,score_reps_2020_5,score_time_2020_5
count,134702.0,134702.000000,134702.000000,117783.000000,134697.000000,134692.000000,134702.000000,134702.000000,118894.000000,116882.000000,116903.000000,134702.000000,134702.000000,134702.000000,134702.000000,71268.000000,67019.000000,65521.000000,61963.000000,49488.000000,45503.000000,44204.000000,42148.000000,42153.000000,35779.000000,32719.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000,134702.000000
mean,2023.0,34.751726,31.839297,15087.215167,178.341465,83.670181,71648.000549,287524.930513,175.733494,123.870048,193.994945,70923.101008,72718.374731,73165.744926,70717.709848,168337.802464,210.158403,142.307382,270.919258,191734.528492,303.786695,739.112614,252.572886,199.010296,220024.944996,500.835050,171.166664,719.692195,174.330054,892.774057,139.081951,539.888302,222.634623,1199.880195,235.574594,1187.292609
std,0.0,7.857439,2.715004,9149.509368,4.039867,6.522887,45354.179799,157647.038954,21.818061,19.350880,35.039180,44416.265384,45921.570161,45459.378833,43239.222649,101621.510195,49.737633,67.961690,190.331728,117611.573377,124.379861,373.602602,245.351308,37.940763,137772.190061,157.750006,78.450610,5.841817,14.957018,33.079739,34.436421,3.031637,34.815243,4.405068,11.773331,59.109607
min,2023.0,16.000000,28.000000,5.000000,161.000000,57.000000,1.000000,107.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,25.000000,1.000000,1.000000,1.000000,101.000000,1.000000,1.000000,1.000000,0.000000,64.000000,1.000000,1.000000,464.000000,1.000000,496.000000,1.000000,354.000000,1.000000,761.000000,1.000000,594.000000
25%,2023.0,29.000000,29.000000,6874.000000,176.000000,79.378759,33675.500000,160640.250000,168.000000,111.000000,169.000000,33679.000000,33827.000000,34185.000000,33690.000000,83048.000000,180.000000,115.000000,156.000000,96335.750000,216.000000,217.000000,135.000000,175.000000,107905.500000,403.000000,125.000000,720.000000,180.000000,900.000000,156.000000,540.000000,240.000000,1200.000000,240.000000,1200.000000
50%,2023.0,34.000000,32.000000,15348.000000,178.778984,84.766796,67476.500000,281619.500000,180.000000,124.000000,192.000000,67552.000000,69200.000000,69406.000000,67534.000000,163242.500000,211.000000,134.000000,167.000000,178699.500000,310.000000,879.000000,145.000000,198.000000,209366.000000,510.000000,139.000000,720.000000,180.000000,900.000000,156.000000,540.000000,240.000000,1200.000000,240.000000,1200.000000
75%,2023.0,40.000000,35.000000,22222.000000,180.000000,86.334657,103790.000000,395173.000000,184.000000,137.000000,217.000000,102871.000000,109393.000000,108625.000000,102939.000000,244744.500000,242.000000,154.000000,369.000000,269955.250000,381.000000,1040.000000,161.000000,225.000000,316621.000000,612.000000,224.000000,720.000000,180.000000,900.000000,156.000000,540.000000,240.000000,1200.000000,240.000000,1200.000000
max,2023.0,54.000000,35.000000,32333.000000,195.580000,127.006015,157742.000000,582777.000000,306.000000,191.000000,375.000000,152041.000000,148203.000000,148144.000000,134389.000000,391859.000000,390.000000,599.000000,719.000000,467118.000000,899.000000,1199.000000,899.000000,365.000000,548388.000000,1030.000000,292.000000,720.000000,180.000000,900.000000,164.000000,540.000000,240.000000,1200.000000,240.000000,1200.000000


In [220]:
df.to_csv('../data/processed/crossfit_db.csv', sep=',', index=False, encoding='utf-8')
